# Updating the Inventory
---
This notebook will provide the code necessary to perform an update to the Biodata Resource Inventory, using trained models.

The steps include:
*   Run new query on EuropePMC
*   Classify new articles
*   Run NER to get resource names for predicted positives
*   Get URLs for predicted positives
*   Gather other metadata



# Setup
---
### Mount Drive

First, mount Google Drive to have access to files necessary for the run:


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/GitHub/inventory_2022/

Run the make target to install Python dependencies.

In [ ]:
! make setup

If you need to download the model checkpoints for the best classifier and NER models, run the cell below. If the `train_and_predict` pipeline was run, then the models are already present.

In [ ]:
# Add command here for downloading models once they're archived

Remove the old output so that Snakemake knows what needs to be done.

**If you have not archived this, do so now!**

In [1]:
!rm data/new_query_results.csv
!rm -rf data/new_paper_predictions/

rm: cannot remove 'new_query_results.csv': No such file or directory


# Running the pipeline
---
Now, we are ready to run the pipeline

## Run it

The following cell will run the pipeline described above. It may take a while, but GPU will speed it up a lot.

In [ ]:
! make update_inventory